In [ ]:
import splitfolders  
import os

fruits_path = "Fruits_Vegetables_Dataset(12000)/Fruits"
vegetables_path = "Fruits_Vegetables_Dataset(12000)/Vegetables"

merged_input = "merged_dataset"

os.makedirs(merged_input, exist_ok=True)

for category in os.listdir(fruits_path):
    os.rename(os.path.join(fruits_path, category), os.path.join(merged_input, category))

for category in os.listdir(vegetables_path):
    os.rename(os.path.join(vegetables_path, category), os.path.join(merged_input, category))

splitfolders.ratio(merged_input, output="output_dataset", seed=42, ratio=(0.8, 0.2))


Copying files: 12000 files [01:26, 139.34 files/s]


In [23]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [28]:
train_dir = "output_dataset/train"
test_dir = "output_dataset/val"

In [29]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images
    transforms.ToTensor(),           # Convert to Tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize
])

In [30]:
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

In [31]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [32]:
class_names = train_dataset.classes
print("Classes:", class_names)

Classes: ['FreshApple', 'FreshBanana', 'FreshBellpepper', 'FreshCarrot', 'FreshCucumber', 'FreshMango', 'FreshOrange', 'FreshPotato', 'FreshStrawberry', 'FreshTomato', 'RottenApple', 'RottenBanana', 'RottenBellpepper', 'RottenCarrot', 'RottenCucumber', 'RottenMango', 'RottenOrange', 'RottenPotato', 'RottenStrawberry', 'RottenTomato']


In [ ]:
class FoodClassifier(nn.Module):
    def __init__(self, num_classes):
        super(FoodClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [34]:
model = FoodClassifier(num_classes=len(class_names)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [35]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

c:\Python311\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/10], Loss: 615.9211, Accuracy: 37.08%
Epoch [2/10], Loss: 386.7937, Accuracy: 59.90%
Epoch [3/10], Loss: 295.9071, Accuracy: 68.82%
Epoch [4/10], Loss: 243.6615, Accuracy: 74.08%
Epoch [5/10], Loss: 200.3153, Accuracy: 78.18%
Epoch [6/10], Loss: 170.7577, Accuracy: 81.49%
Epoch [7/10], Loss: 147.2955, Accuracy: 83.68%
Epoch [8/10], Loss: 125.4060, Accuracy: 85.82%
Epoch [9/10], Loss: 114.4758, Accuracy: 87.04%
Epoch [10/10], Loss: 102.7707, Accuracy: 88.17%


In [36]:
torch.save(model.state_dict(), "food_classifier.pth")
print("Model saved!")

Model saved!


In [37]:
model.eval()
correct = 0
total = 0

In [38]:
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 83.06%


In [39]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os

In [40]:
class FoodClassifier(torch.nn.Module):
    def __init__(self, num_classes):
        super(FoodClassifier, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.fc1 = torch.nn.Linear(64 * 32 * 32, 128)
        self.fc2 = torch.nn.Linear(128, num_classes)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [44]:
class_names = ['Apple', 'Banana','Bell Pepper','Carrot','Cucumber','Mango','Orange','Potato','Strawberry','Tomato','Rotten Apple', 'Rotten Banana','Rotten Bell Pepper','Rotten Carrot','Rotten Cucumber','Rotten Mango','Rotten Orange','Rotten Potato','Rotten Strawberry','Rotten Tomato']  # Update with your classes

In [45]:
num_classes = len(class_names)
model = FoodClassifier(num_classes=num_classes).to(device)
model.load_state_dict(torch.load("food_classifier.pth", map_location=device))
model.eval()

FoodClassifier(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=20, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

In [46]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to match model input
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize
])

In [ ]:
def predict_food(image_path):
    image = Image.open(image_path).convert("RGB") 
    image = transform(image).unsqueeze(0).to(device) 

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    return class_names[predicted.item()]

In [51]:
image_path = "output_dataset/val/FreshOrange/freshOrange (11).jpg"
prediction = predict_food(image_path)
print(f"Predicted food item: {prediction}")

Predicted food item: Orange
